In [24]:
import custom_utils as cu
from ultralytics import YOLO
import numpy as np
import cv2

In [3]:
# step 0: get images 
p, images = cu.get_images("/home/jay/module/ai_app/self_driving_cars/aai-selfdriving-cars/dataset/traffic_light/green", 100)

In [4]:
# step 1: analyze images with yolo
model = YOLO("yolov8n.pt")
results = model.predict(images, save=True, save_txt = True)


0: 640x640 1 traffic light, 1: 640x640 2 traffic lights, 2: 640x640 1 traffic light, 3: 640x640 2 traffic lights, 4: 640x640 1 traffic light, 5: 640x640 2 traffic lights, 6: 640x640 1 traffic light, 1 bottle, 7: 640x640 (no detections), 8: 640x640 1 traffic light, 9: 640x640 2 traffic lights, 10: 640x640 2 persons, 2 cars, 1 motorcycle, 1 traffic light, 11: 640x640 1 traffic light, 12: 640x640 1 traffic light, 13: 640x640 1 person, 3 cars, 14: 640x640 (no detections), 15: 640x640 1 traffic light, 16: 640x640 6 persons, 6 cars, 1 motorcycle, 3 traffic lights, 17: 640x640 (no detections), 18: 640x640 1 traffic light, 19: 640x640 2 traffic lights, 20: 640x640 1 traffic light, 21: 640x640 1 traffic light, 22: 640x640 1 traffic light, 23: 640x640 1 traffic light, 1 refrigerator, 24: 640x640 2 traffic lights, 1 apple, 25: 640x640 1 person, 3 cars, 2 traffic lights, 26: 640x640 (no detections), 27: 640x640 1 traffic light, 28: 640x640 1 traffic light, 29: 640x640 1 traffic light, 30: 640x640

In [43]:
# todo 
# drop similar boxes is requiered becaus some times yolo identify the same object with a different bounding box
def drop_similar_boxes(input_list, threshold):
    def are_elements_similar(elem1, elem2):
        x1_diff = abs(elem1[0] - elem2[0])
        y1_diff = abs(elem1[1] - elem2[1])
        x2_diff = abs(elem1[2] - elem2[2])
        y2_diff = abs(elem1[3] - elem2[3])
        return x1_diff < threshold or y1_diff < threshold or x2_diff < threshold or y2_diff < threshold

    unique_elements = []
    for elem in input_list:
        is_similar = False
        for unique_elem in unique_elements:
            if are_elements_similar(elem, unique_elem):
                is_similar = True
                break
        if not is_similar:
            unique_elements.append(elem)
    return unique_elements
"""
input_list = [[1.0, 2.0, 3.0, 4.0], [1.2, 2.2, 3.0, 4.0], [5.0, 6.0, 7.0, 8.0], [1.1, 2.1, 3.1, 4.1]]
threshold = 0.2
filtered_list = drop_similar_boxes(input_list, threshold)
print(filtered_list)
"""

'\ninput_list = [[1.0, 2.0, 3.0, 4.0], [1.2, 2.2, 3.0, 4.0], [5.0, 6.0, 7.0, 8.0], [1.1, 2.1, 3.1, 4.1]]\nthreshold = 0.2\nfiltered_list = drop_similar_boxes(input_list, threshold)\nprint(filtered_list)\n'

In [25]:
def average_resize(images):
    dia_len = [(i.shape[1]**2 + i.shape[2])**0.5 for i in images]
    avr = int(np.average(dia_len))
    for idx, i in enumerate(images):
        i = cv2.resize(i, (avr,avr))
        cv2.imwrite("/home/jay/module/ai_app/self_driving_cars/aai-selfdriving-cars/test_out/resize/" + str(idx) + ".jpg", i)

In [13]:
# step 2: extract all traffic lights from all analyzed images and create for each an new image
# Process results list
out_path = '/home/jay/module/ai_app/self_driving_cars/aai-selfdriving-cars/test_out/'
image_name = 'traffic_light_green'
file_type = '.jpg'

counter = 0
extracted_images = []
for j, result in enumerate(results):
    for i,c in enumerate(result.boxes.cls.numpy()):
        # find traffic light boxes
        points = result.boxes.xyxyn.numpy()[i]
        if c == 9:
            path = out_path + image_name + str(counter) + file_type
            new_image = cu.extract_rectangle_from_image(images[j], points, save=False, output_path = path) 
            extracted_images.append(new_image)
            counter += 1
            
# resize imgaes
average_resize(extracted_images)

# step 3: check images manually and label them